# Google Play Store Twitter Sentiment Analysis
Data provided by [Kaggle](https://www.kaggle.com/lava18/google-play-store-apps). In addition Twitter will be analyzed for any further data on the app itself.

A coulple questions come to mind:
1. Is there really a corelation between the number of times an app was downloaded and the Twitter sentiment.
2. Try to find from the top 10000 apps (based on installs) how each install correlates to the positve and negative sentiments on Twitter.
3. Try to find which genre of apps brings the most positive and negative sentiment out of the top 10 downloads from the play store.


For the purpose of the second task **Syuzhet** will be used. **Syuzhet** get's the sentiment into 10 emotions: anger, anticipation, disgust, fear, joy, sadness, surprise, trust, negative and positive.
The notebook will examine only the positive and negative sentiments for each first 100 tweets for each app.

But first let's start with loading the libraries:

In [7]:
library(ggplot2)
library(Amelia)
library(dplyr)
library(rjson)
library(twitteR)
library(syuzhet)

set.seed(42)

## 1. Loading and preprocessing the data.

### 1.1 Loading the dataset.

In [8]:
df <- data.frame(read.csv('data/googleplaystore.csv'))

In [18]:
# Displaying the head
head(df)

App,Category,Rating,Reviews,Size,Installs,Type,Price,Content.Rating,Genres,Last.Updated,Current.Ver,Android.Ver
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up


### 1.2 Configuring the twitter connection

In [10]:
twitterCreds <- fromJSON(file = "data/twitter_access.json")

In [11]:
setup_twitter_oauth(twitterCreds$consumer_key, 
                    twitterCreds$consumer_secret, 
                    access_token=twitterCreds$access_token, 
                    access_secret=twitterCreds$access_token_secret)

[1] "Using direct authentication"


In [17]:
# code snippets to remind me how its done.
# soccer.tweets <- searchTwitter("soccer", n=2000, lang="en")
# soccer.tweets.df <- twListToDF(soccer.tweets)
# get_nrc_sentiment(soccer.tweets.df$text)

### 1.3 Getting the bearings of the datasets.
First - let's get the dimensions and structure of the dataset.

In [28]:
dim(df)

[1] 10841    13

In [19]:
str(df)

'data.frame':	10841 obs. of  13 variables:
 $ App           : Factor w/ 9660 levels "- Free Comics - Comic Apps",..: 7206 2551 8970 8089 7272 7103 8149 5568 4926 5806 ...
 $ Category      : Factor w/ 34 levels "1.9","ART_AND_DESIGN",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Rating        : num  4.1 3.9 4.7 4.5 4.3 4.4 3.8 4.1 4.4 4.7 ...
 $ Reviews       : Factor w/ 6002 levels "0","1","10","100",..: 1183 5924 5681 1947 5924 1310 1464 3385 816 485 ...
 $ Size          : Factor w/ 462 levels "1,000+","1.0M",..: 55 30 368 102 64 222 55 118 146 120 ...
 $ Installs      : Factor w/ 22 levels "0","0+","1,000,000,000+",..: 8 20 13 16 11 17 17 4 4 8 ...
 $ Type          : Factor w/ 4 levels "0","Free","NaN",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Price         : Factor w/ 93 levels "$0.99","$1.00",..: 92 92 92 92 92 92 92 92 92 92 ...
 $ Content.Rating: Factor w/ 7 levels "","Adults only 18+",..: 3 3 3 6 3 3 3 3 3 3 ...
 $ Genres        : Factor w/ 120 levels "Action","Action;Action & Adventure",..: 10 13 10 10 

**Structure summary**:  
From the structure it seems that some columns, that one would expect to be of type *numerical* are actually *categoricals*. The only one that would be true is *Reviews* as the column indicates the number of reviews an app is getting. In addition *Size* also could be translated to a numerical value as there are 462 levels. On the other hand it seems that the column *Installs* is aggregated in 22 levels and is indeed a categorical value. Same goes for the *Price*. The *Last.Updated* column seems like a date-time.

In [26]:
unique(df$Reviews)

[1] 159      967      87510    215644   167      178      36815    13791   
   [9] 121      13880    8788     44829    4326     1518     55       3632    
  [17] 27       194216   224399   450      654      7699     61       118     
  [25] 192      20260    203      136      223      1120     227      5035    
  [33] 1015     353      564      8145     36639    158      591      117     
  [41] 176      295221   2206     26       174531   1070     85       845     
  [49] 367      1598     284      17057    129      542      10479    805     
  [57] 1403     3971     534      7774     38846    2431     6090     295     
  [65] 190      40211    356      52530    116986   1379     271920   736     
  [73] 7021     197      737      3574     994      197136   142      15168   
  [81] 2155     138      5414     21777    348      250      13372    7880    
  [89] 3617     4806     65786    31433    5097     1754     2680     1288    
  [97] 18900    49790    1150     1739     32090    2225     4369     8572    
 [105] 964      42050    104      17934    601      36       187      182     
 [113] 30       134      74       113715   3595     9315     75       38      
 [121] 26834    119      2277     2280     184      9        77       35      
 [129] 364      18       473      66       3871     257      62       2914724 
 [137] 1857     4478     577550   814080   246315   454060   155446   418     
 [145] 22486    203130   1435     116507   1433233  90468    860      363934  
 [153] 87873    17506    1862     2084     47303    19080    161      85842   
 [161] 7831     91615    4620     21336    26875    1778     2709     64513   
 [169] 8342     527      1322     1680     2739     1065     233757   2       
 [177] 51269    30105    156      114      341157   16129    674730   1254730 
 [185] 85185    32584    217730   70991    1002861  16589    148945   4458    
 [193] 62272    8941     46353    1279184  88073    67000    159872   30847   
 [201] 188841   11622    95912    4034     45964    14955    6903     31614   
 [209] 23055    19023    207372   1225     380837   10600    74359    822     
 [217] 80805    2287     4162     14760    23243    8978     42492    286897  
 [225] 103755   46505    11442    10295    296      29313    51507    1802    
 [233] 1383     23175    5868     2111     5448     4159     20815    78662   
 [241] 7149     3079     5800     6989     16422    108741   624      1661    
 [249] 97702    308      5211     1058     78172    80804    1002859  51510   
 [257] 413      1013635  24005    57106    2249     516      834      1010    
 [265] 238970   302      438      73       39       144      2181     93965   
 [273] 1446     12088    314      25671    15194    22551    29839    279     
 [281] 564387   1330     1677     757      115      125      9952     18814   
 [289] 21       15       51981    3596     1006     5968     4895     56642847
 [297] 69119316 125257   9642995  1429035  4604324  3419249  11334799 158679  
 [305] 3075028  4187998  659395   4785892  66602    30209    36901    5149854 
 [313] 192948   99559    437674   13698    2473509  20769    36880    171031  
 [321] 63543    45487    615381   2451083  33053    5387333  3648120  136662  
 [329] 42370    781810   3128250  2083237  541389   46702    2939     13761   
 [337] 258556   40751    17712922 25021    27187    56646578 4785988  122498  
 [345] 11334973 3419433  132014   83239    594728   5387446  10484169 3128509 
 [353] 2451093  2876500  28238    335646   350154   349384   346982   244863  
 [361] 10790289 330761   37320    12842860 2546527  36893    15880    346980  
 [369] 9643041  3075118  2264916  2473693  42925    5150471  2511130  3648480 
 [377] 17714850 13100    27156    55098    1133501  12578    10965    18247   
 [385] 190613   125232   72065    27540    104990   177703   17529    177263  
 [393] 237468   32254    483565   552441   60308    457283   93825    32283   
 [401] 15287    205739   14873    7820209  9498     4